# Автокорректор ошибок на Python

**Вначале немного теории.**

Перед нами стояла следующая задача: изучить алгоритмы, основанных на сравнении слова со списком правильных слов. Запрограммировать эти алгоритмы.

Для алгоритмов сравнения со словарём рекомендую использовать: Расстояние Жаккара и Расстояние Левенштейна.

## 1 Часть. Сравнение расстояний Жаккара и Левенштейна.

Создадим две рекомендательные системы правописания, которые смогут принимать входные данные пользователя и рекомендовать правильно написанное слово, одно при помощи расстояния Жаккара, а другое - Левенштейна.

In [ ]:
import nltk
from nltk.corpus import words
from nltk.metrics.distance import (
edit_distance,
jaccard_distance,
)
from nltk.util import ngrams
import pandas

correct_spellings = #список правильных слов
spellings_series = pandas.Series(correct_spellings) # проиндексированный список этих словv


**Метрика: расстояние Жаккара**

Расстояние Жаккара является мерой того, насколько непохожи два набора сдлва, и может быть найдено как дополнение к индексу Жаккара (т. е. расстояние Жаккара = 100% - индекс Жаккара).

Рассматриваемые слова будут итеративно сравниваться с каждым словом в spellings_series.

Spellings создаст список возможных слов на основе первой буквы строки (предполагается, что первая буква напечатана правильно).

Далее, Distances будет итеративно вычислять соответствующие расстояния Жаккара для слов в написании с помощью встроенной функции jaccard_distance.

Наконец, closest даст результирующее слово с лучшим соответствием через функцию min на расстояниях. Затем это слово будет добавлено в список результатов, и этот список будет возвращен после завершения работы функции.



In [ ]:
def jaccard(entries, gram_number):

outcomes = []
for entry in entries: 
    for loopspellings = spellings_series[spellings_series.str.startswith(entry[0])]
    distances = ((jaccard_distance(set(ngrams(entry, gram_number)), set(ngrams(word, gram_number))), word) for word in spellings)
    closest = min(distances)
    outcomes.append(closest[1])
return outcomes

In [ ]:
def JDreco(entries=['ево', 'чуство', 'юмара']):
 #finds the closest word based on jaccard distance
    return jaccard(entries, 3)
print(JDreco())

**Метрика: расстояние Левенштейна**

Этот метод оценивает, насколько непохожи две строки, на основе минимального количества операций, необходимых для преобразования одной строки в другую.

Функция итеративно сравнивает записи со списком правильных слов и возвращает Расстояние Левенштейна Затем слово с наименьшим расстоянием будет считаться наиболее правильным словом.

In [ ]:
def editreco(entries=['мома', 'мфла', 'рму']):

outcomes = []
for entry in entries:
    distances = ((edit_distance(entry, word), word) for word in correct_spellings)
    closest = min(distances)
    outcomes.append(closest[1])
return outcomes


print(editreco())

Теперь дававйте сравним результаты двух работ

In [ ]:
userinput = []
for i in range(0,3):
    word = input("Я лублю пильмени: ")
    userinput.append(word)
JDreco(userinput)
editreco(userinput)

**Место для анализа результатов**

# 2часть. Модель Bag of words

In [1]:
# Импортируй и властвуй
%pylab inline
import re
import math
import string
from collections import Counter
import requests

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Прежде чем что-то делать со словами, надо эти слова откуда-то взять. Мы сами сделали свой текстовый файл, text.txt, в котором записано много слов из таких произведений как: "Тихий Дон", "Преступление и наказание", "Котлован", "Каштанка", "Капитанская дочка", а также *дополнить*. 


In [2]:
import codecs
with codecs.open('text.txt', 'r', encoding = 'utf-8') as file:
    TEXT = file.read().replace('\n', ' ') # для текста, в котором слова разделены '\n'


UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 899: character maps to <undefined>

In [ ]:
with codecs.open('russian.txt', 'r', encoding = 'windows 1251') as file:
    TEXT2 = file.read().replace('\n', ' ') # для текста, в котором слова разделены '\n'


Затем нужно этот текст разбить на слова (необходимо выполнить токенизацию). 

In [ ]:
def tokens(text):
    #Возвращает список токенов (подряд идущих буквенных последовательностей) в тексте. 
       #Текст при этом приводится к нижнему регистру.
    return re.findall(r'[а-ё]+', text.lower()) 

In [ ]:
WORDS = tokens(TEXT)
print('Количество слов в словаре: ' + len(WORDS))
print( 'Количество неповторяющихся слов в словаре: '+ len(set(WORDS)))

WORDS2 = tokens(TEXT2)
print(len(WORDS2))

Хочу заметить, что сейчас слова появляются в нашем списке в том порядке, как они располагались в файле

***Модель: Мешок слов (aka Bag of Words)***

Мы создали список `WORDS` - список слов в том порядке, как они следуют в `TEXT`. Мы можем использовать этот список в качестве порождающей модели текста. Язык - очень сложная штука и мы создаем крайне упрощенную модель языка, которая может ухватить часть этой сложной структуры. 

В модели мешка слов, мы полностью игнорируем порядок слов, зато соблюдаем их частоту. Представить это можно себе так: вы берете все слова текста и забрасываете их в мешок. Теперь, если вы хотите сгенерировать предложение с помощью этого мешка, вы просто трясете его(слова там перемешиваются) и достаете указанное количество слов по одному (мешок непрозрачный, так что слоа вы достаете наугад). Почти наверное полученное предложение будет грамматически некорректным, но слова в этом предложении будут в +- правильной пропорции (более частые будут встречаться чаще, более редкие - реже). Вот функция, которая сэмплирует предложение из $n$ слов с помощью нашего мешка:

In [ ]:
import random
def sample(bag, n=10):
    "Выборка случайного предложения из n слов из модели, описанной мешком слов."
    return ' '.join(random.choice(bag) for _ in range(n))

In [ ]:
sample(WORDS)

Другое представление мешка слов - **`Counter`**. Это словарь, состоящий из пар 

`{'слово': кол-во вхождений слова в текст}`. 

Например:

In [ ]:
Counter(tokens('Между нами провода, Города да да да. Я сказал иди сюда, И ты сказала: «Да, да, да..»'))

Давайте завернем в Counter наш список слов WORDS и посмотрим, что получится:


In [ ]:
COUNTS = Counter(WORDS)

print('Десять самых часто встречающихся в тексте слов ' + COUNTS.most_common(10))

In [ ]:
for w in tokens('самые редкие слова: Крыжить, Михрютка, Драдедамовый '):
    print(COUNTS[w], w)

----------------------------------------------------------------------------
![Джордж Ципф](http://www.factroom.ru/facts/wp-content/uploads/2013/12/224.jpg " ")

В 1935, лингвист Джордж Ципф отметил, что в любом большом тексте $n - $тое наиболее часто встречающееся слово появляется с частотой ~ $ 1/n$ от частоты наиболее часто встречающегося слова. Это наблюдение получило название Закона Ципфа, несмотря на то, что Феликс Ауэрбах заметил это еще в 1913 году. Если нарисовать частоты слов, начиная от самого часто встречающегося, на log-log-графике, они должны приблизительно следовать прямой линии, если закон Ципфа верен. 

In [ ]:
M = COUNTS['и'] 
yscale('log'); 
xscale('log'); 
title('Частота n-того наиболее частого слова и линия 1/n.')
plot([c for (w, c) in COUNTS.most_common()])
plot([M/i for i in range(1, len(COUNTS)+1)]);

**Анализ полученных результатов:**

----------------------------------------------------------------------------
*дополнительное, незнаю, что с этим делать*
Но чем плох Bag of words? Тем, что для него кошк|у| != кошк|а| - два разных слова. Как это побороть? А что если мы вырежем все окончания и суффиксы, одним словом сделаем **стемминг Портера**. Тем самым мы будем считать что слово одно, а его форм много, но они уже не считаются за разные слова.

In [ ]:
#что-то реализую
WORDS += WORDS2
#WORDS = [w for w in WORDS if len(w) > 2]

Но тогда возникает проблема, существуют слова, для которых стемминг не приведет к одному и тому же слову, например "шел" и "идти". Тогда на помощь приходят морфологические анализаторы, которые приводят слово к начальной форме (происходит лемматизация)

In [ ]:
from pymystem3 import Mystem
text = "Красивая мама красиво мыла раму" 
m = Mystem()
lemmas = m.lemmatize(text)
print(''.join(lemmas))

In [ ]:
import pymystem3
m1=pymystem3.Mystem()

m1.analyze('шел')

In [ ]:
Но в таком случае возникает проблема, что делать с "стекло" или "эти типы стали есть в прокатном цехе".
Морфологические анализаторы иногда делают ошибки и без контекста им не разобраться.
Но pymystem вроде как +- справляются
----------------------------------------------------------------------------

# 3 часть. Проверка Правописания

Применим наивный подход: найдем все кандидаты, достаточно близкие к w и выберем более близкое слово, если проверки на близость недостаточно, берем из подходящих слово с максимальной частотой из WORDS. 

Сейчас мы будем измерять близость с помощью расстояния Левенштейна: минимального необходимого количества удалений, перестановок, вставок, и замен символов, необходимых чтобы одно слово превратить в другое. 
`Расстояние Левенштейна:
Исходное слово --> поиск
Вставка --> происк
Удаление --> п_иск
Замена --> поеск`

Тогда остается определить функцию *correct(w):*

In [ ]:
def correct(word):
    "Поиск лучшего исправления ошибки для данного слова."
    # предрассчитать edit_distance==0, затем 1, затем 2; в противном случае оставить слово "как есть".
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

функция `edits2` iлегко получается из функции `edits1`:

In [ ]:
def known(words):
    "Вернуть подмножество слов, которое есть в нашем словаре."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    "Вернуть все строки, которые находятся на edit_distance == 0  (т.е., просто само слово)."
    return {word}

def edits2(word):
    "Вернуть все строки, которые находятся на edit_distance == 2 ."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

Функция `edits1(word)` должна возвращать множество слов, находящихся на расстоянии `edit_distance == 1`. Например для слова "киты" это множество будет включать слова "коты" (замена и на o) и слово "ккиты" (вставка к),  а также "икты" (перемена к и и местами); после чего может быть применена функция known для фильтрации и выбора подходящих кандидатов). 
Как же нам получить их? Например можно разбить исходное слово на пару всеми возможными способами (каждое разбиение даст нам пару "слов"), (a, b), первая часть - до места разбиения, а вторая - после, и в каждом месте разбиения можно: удалить, поменять местами, заменить или вставить букву:

In [ ]:
def edits1(word):
    "Возвращает список всех строк на расстоянии edit_distance == 1 от word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    "Возвращает список всех возможных разбиений слова на пару (a, b)."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьяюя'

In [ ]:
len(alphabet)

In [ ]:
print(edits0('атец'))
print(edits1('атец'))
print(len(edits2('атец')))

In [ ]:
tokens('Ана ни зочит в шшколу.')

In [ ]:
list(map(correct, tokens('Ана ни зочит в шшколу.')))

Выходные данные можно сделать покрасивее, сохранив регистр

In [ ]:
def correct_text(text):
    "Исправить все слова с опечатками в тексте."
    return re.sub('[а-ёА-Ё]+', correct_match, text)

def correct_match(match):
    "Исправить слово word в match-группе, сохранив регистр: upper/lower/title."
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    "Возвращает функцию регистра по тексту: upper, lower, title, или str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

***Насколько дорого превращать одно слово в другое?***

Динамическое программирование позволяет разбить задачу на подзадачи, решив которые можно скомпоновать финальное решение. Мы будем пытаться превратить строку $source[0..i]$ в строку $target[0..j]$, мы сосчитаем все возможные комбинации подстрок $substrings[i, j]$ и рассчитаем их *edit_distance* до нашей исходной. Мы будем сохранять результаты в таблицу и переиспользовать их для расчета дальнейших изменений.


In [ ]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: строка-исходник
        target: строка, в которую мы должны исходник превратить
        ins_cost: цена вставки
        del_cost: цена удаления
        rep_cost: цена замены буквы
    Output:
        D: матрица размера len(source)+1 на len(target)+1 содержащая минимальные расстояния edit_distance
        med: минимальное расстояние edit_distance (med), необходимое, 
        чтобы превратить строку source в строку target
    '''
    # стоимость удаления и вставки = 1
    m = len(source)
    n = len(target)

    # Заткнем нашу матрицу нулями
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # Заполним первую колонку
    for row in range(1,m+1): 
        D[row,0] = D[row-1,0] + del_cost
        
    # Заполним первую строку
    for col in range(1,n+1): 
        D[0,col] = D[0,col-1] + ins_cost
        
    # Теперь пойдем от 1 к m-той строке
    for row in range(1,m+1): 
        
        # итерируемся по колонкам от 1 до n
        for col in range(1,n+1):
            
            # r_cost - стоимость замены
            r_cost = rep_cost
            
            # Совпадает ли буква исходного слова из предыдущей строки
            # с буквой целевого слова из предыдущей колонки, 
            if source[row-1] == target[col-1]:
                # Если они не нужны, то замена не нужна -> стоимость = 0
                r_cost = 0
                
            # Обновляем значение ячейки на базе предыдущих значений 
            # Считаем D[i,j] как минимум из трех возможных стоимостей (как в формуле выше)
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
          
    # установить edit_distance в значение из правого нижнего угла
    med = D[m,n]
    

    return D, med

In [ ]:
import pandas as pd

source =  'кроты'
target = 'киты'
matrix, min_edits = min_edit_distance(source, target)

print("Расстояние: ",min_edits, "\n")

idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

## 4 часть. От счетчика слов к вероятностям последовательностей слов. Разбиение на сегменты

Нам нужно научиться подсчитывать вероятности слов, $P(w)$. Делать мы это будем с помощью функции `pdist`, которая на вход принимает Counter (мешок слов) и возвращает функцию, выполняющую роль вероятностного распределения на множестве всех возможных слов. В вероятностном распределении вероятность каждого слова лежит между 0 и 1, и сложение вероятностей всех слов дает 1.

In [ ]:
def pdist(counter):
    "Превращает частоты из Counter в вероятностное распределение."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

P = pdist(COUNTS)

In [ ]:
for w in tokens('То мать пирогов напечет, то бабушка с булочками приедет'):
    print(P(w), w)

Что такое вероятность последовательности слов? 
Используем определение совместной вероятности:
$P(w_1 ... w_n) = P(w_1)*P(w_2|w_1)*P(w_3|w_1w_2)...*...P(w_n|w_1...w_n-1)$

Модель мешка слов подразумевает, что каждое слово из мешка достается независимо от других. Это дает нам упрощенную аппроксимацию:
$P(w_1 ... w_n) = P(w_1)*P(w_2)*P(w_3)...*...P(w_n)$

Известный статистик Джордж Бокс сказал Все модели неверны, но некоторые полезны.

Как же нам посчитать $P(w_1 ... w_n)$? Мы будем использовать другое название `Pwords` вместо `P`, и посчитаем ее как произведение индивидуальных вероятностей:

In [ ]:
def Pwords(words):
    "Вероятности слов, при условии, что они независимы."
    return product(P(w) for w in words)

def product(nums):
    "Перемножим числа."
    result = 1
    for x in nums:
        result *= x
    return result

In [ ]:
tests = ['тест', 
         'дом',
         'Крыжить']

for test in tests:
    print(Pwords(tokens(test)), test)

кажется, присвоить последнюю вероятность 0, неправильно; Она просто должна быть маленькой. К этому вернемся попозже. Ну а другие вероятности кажутся +- адекватными.

## Разбиение слов на сегменты
Исправляем ошибки, когда слова пишутся слитно (по ошибке или нет)


**Подход 1:** Перенумеруем все возможные разбиения и выберем то, у которого максимальная Pwords

Вопрос: Как выбрать количество сегментов для строки длины n?

**Подход 2:** Делаем одно разбиение - на первое слово и все остальное. Если предположить, что слова независимы, можно максимизировать вероятность первого слова + лучшего разбиения оставшихся букв.


`assert segment('фотоальбом') == ['фото', 'альбом']
segment('фотоальбом') ==
   max(Pwords(['ф'] + segment('отоальбом')),
       Pwords(['фо'] + segment('тоальбом')),
       Pwords(['фот'] + segment('оальбом')),
       Pwords(['фото'] + segment('альбом')),
       ...
       Pwords(['фотоальбом'] + segment('')))`

Чтобы сделать это эффективным, нужно избежать слишком большого числа пересчетов оставшейся части слова. Это можно сделать c помощью кэширования. Кроме того, для первого слова не обязательно брать все возможные варианты разбиений - мы можем установить максимальную длину. Какой она должна быть? Чуть большей, чем длина самого длинного слова, которое мы видели.

In [ ]:
def memo(f):
    "Запомнить результаты исполнения функции f, чьи аргументы args должны быть хешируемыми."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [ ]:
max(len(w) for w in COUNTS)

In [ ]:
def splits(text, start=0, L=20):
    "Вернуть список всех пар (a, b); start <= len(a) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

In [ ]:
print(splits('слово'))
print(splits('длинныйтекст', 3, 7))

In [ ]:
@memo
def segment(text):
    "Вернуть список слов, который является наиболее вероятной сегментацией нашего текста."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

In [ ]:
segment('сдырочкой')

In [ ]:
decl = ('предложениеэтосложнаясинтаксическаяконструкция')

In [ ]:
print(segment(decl))

**Анализ:**

## 5 часть. Анализ с помощью биграмм


---------------------------------------------------------------------------
Можно перейти к анализу пар последоваительных слов, не ожидая, что вероятности слишком часто будут обнуляться (представьте себе, сколько в языке может быть грамматически корректных сочетаний из двух слов).

In [ ]:
def load_counts(text, sep='\t'):
    """Возвращает Counter, полученный из пар ключ-значение,в каждой строке файла."""
    C = Counter()
    for i in [l.split('\t') for l in text.split('\n')][:-1]:
        key, count = i
        C[key] = int(count)
    return C

In [ ]:
COUNTS1 = список слов с частотой их встречания
COUNTS2 = список слов с частотой их встречания

P1w = pdist(COUNTS1)
P2w = pdist(COUNTS2)

In [ ]:
print(len(COUNTS1), sum(list(COUNTS1.values()))/1e9)
print(len(COUNTS2), sum(list(COUNTS2.values()))/1e9)

In [ ]:
COUNTS2.most_common(30)

**Сегментация с помощью биграмм**

Чуть менее неправильная аппроксимация:
$P(w_1...w_n)=P(w_1)*P(w_2|w_1)*P(w_3|w_2)*...*...P(w_n|w_n-1)$
Эта штука называется биграммной моделью. Представьте, что вы взяли текст, достали из него все возможные пары подряд идущих слов и положили каждую пару в мешок, промаркированный ПЕРВЫМ словом из пары. После этого, чтобы сгенерировать кусок текста, мы берем первое слово из исходного мешка слов , а каждое следующее слово вынимаем из соответствующего мешка биграмм.

Начнем с определения вероятности текущего слова при условии данного предыдущего слова из Counter:

Отмечу, что для английского языка биграммная модель будет выглядеть так:
$P(w_1...w_n)=P(w_1)*P(w_2|w_1)*P(w_3|w_2)*...*...P(w_n|w_n-1)$ условная вероятность слова при условии предыдущего слова определяется так:
$P(w_n|w_n-1)=P(w_n-1w_n)/P(w_n-1)$

In [ ]:
def Pwords2(words, prev='<S>'):
    "Вероятность последовательности слов с помощью биграммной модели(при условии предыдущего слова)."
    return product(cPword(w, (prev if (i == 0) else words[i-1]) )
                   for (i, w) in enumerate(words))

# Перепишем Pwords на большой словарь P1w вместо Pword
def Pwords(words):
    "Вероятности слов при условии их независимости."
    return product(P1w(w) for w in words)

def cPword(word, prev):
    "Условная вероятность слова при условии предыдущего."
    bigram = prev + ' ' + word
    if P2w(bigram) > 0 and P1w(prev) > 0:
        return P2w(bigram) / P1w(prev)
    else: # если что-то не встретилось, поставим среднее между P1w и 0
        return P1w(word) / 2

In [ ]:
print(Pwords(tokens('Он приехал в город')))
print(Pwords2(tokens('Приехал он в город')))
print(Pwords2(tokens('Город в он приехал')))

Чтобы сделать `segment2`, скопируем `segment`, добавим в аргументы предыдущий токен, а вероятности будем считать с помощью `Pwords2` вместо `Pwords`

In [ ]:
@memo 
def segment2(text, prev='<S>'): 
    "Возвращает наилучшее разбиение текста, используя статистику биграмм." 
    if not text: 
        return []
    else:
        candidates = ([first] + segment2(rest, first) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=lambda words: Pwords2(words, prev))


In [ ]:
print(segment2('фотоальбом'))
print(segment2('поехалаонанет'))


**Анализ Биграммной модели:  вроде бы лучше, но не очень.**

## 6 часть.Валидация

До настоящего момента мы пытались интуитивно оценить результаты нашей работы. Тем не менее, никаких численных оценок качества мы пока не получили. Важно понимать, что без четких метрик слова "плохо"/"хорошо" не имеют никакого смысла. Более того - мы даже не можем четко ответить, было ли наше обновление модели в лучшую сторону или худшую. 

Обычно при построении неких прогностических моделей данные разбиваются на три части:

* **Обучающая выборка:** то, что мы использовали для создания модели исправления ошибок; У нас это был файл text.txt.
* **Тестовая выборка:** набор данных, который можно использовать для оценки качества вашей модели по ходу разработки.
* **Валидационная выборка:** Набор данных, который мы используем для оценки работы программы после того как программа готова. Тестовая выборка для этого быть использована не может. 

Для нашей программы обучающая выборка - словарь слов с частотами, а тестовая выборка - набор примеров типа "фотоальбом". 

Сделаем валидационную выборку.

In [ ]:
def test_segmenter(segmenter, tests):
    "Оценка сегментатора на тестовых данных; вывести на печать ошибки; вернуть долю верно разбитого."
    return sum([test_one_segment(segmenter, test) 
               for test in tests]), len(tests)

def test_one_segment(segmenter, test):
    words = tokens(test)
    result = segmenter(''.join(words))
    correct = (result == words)
    if not correct:
        print('expected', words)
        print('got     ', result) 
    return correct

proverbs = ("""Унас под крыльцом живут ежи. По вечерам вся смья выходит гулять.""".splitlines())


In [ ]:
test_segmenter(segment, proverbs)

## 7часть. Сглаживание

Проблема в том, что если в последовательности вероятность одного из слов - 0, то вся вероятность бы обнулилась (мы же считаем произведение). Кажется, что это слишком строгое условие; Словарь не идеален и точно существуют реальные слова, которых мы не увидели. Давайте не будем все сразу обнулять. 
Например точно должна быть оценка вероятности того, что слово настоящее. Скажем, "сверхзвуковой" явно должно быть более вероятным чем "рыбркле" и должно иметь бОльшую вероятность.

Проблему можно побороть, присвоив таким "не встретившимся" словам ненулевую вероятность. Еще более важным этот пункт становится при переходе к токенам из нескольких слов (биграммам, например), потому что чем больше слов в токене, тем больше вероятность, что какой-то реальный токен в нашей обучающей выборке отсутствует.

Нашу модель можно представить в виде забора вероятностей, где столбик равен вероятности слова/токена, которое/который в выборке было/был, и равен 0, если слова/токена в выборке не было; Мы хотим сгладить наше распределение вокруг этих пиков, чтобы модель давала какой-то ответ вне зависимости от наличия или отсутствия слова в корпусе. Этот процесс и называется сглаживанием.

**Место для анекдота про Лапласа**

Однажды французского математика Лапласа спросили: "Какова вероятность того, что Солнце завтра взойдет?". Из данных, что оно из  ближайших дней взошло n раз следует оценка максимального правдоподобия $n/n = 1$. Но Лапласу хотелось чуть сбалансировать оценку на шанс того, что завтра Солнце может и не взойти, поэтому он дал оценку $(n+1)/(n+2)$.

То, что мы знаем, ограничено, а то, чего мы не знаем,-бесконечно
— Пьер Симон Лаплас, 1749-1827

In [ ]:
def pdist_additive_smoothed(counter, c=1):
    """Вероятность слова, при условии данных из Counter'a.
    добавляем c к частоте каждого слова + слово 'unknown'."""
    N = sum(list(counter.values()))          # суммарное кол-во слов
    Nplus = N + c * (len(counter) + 1) # кол-во слов + сглаживание
    return lambda word: (counter[word] + c) / Nplus 

P1w = pdist_additive_smoothed(COUNTS1)

In [ ]:
P1w('сверхзвуковой')

Теперь еще одна проблема ... у нас появились незнакомые слова с ненулевой вероятностью. А что если 10-12 - приемлемая вероятность для слов нашего текста: то есть, если я читаю новый текст, вероятность того, что следующее слово мне незнакомо, может быть порядка 10-12. Но если я случайно генерирую 20-буквенный последовательности, вероятность того, что одна из них будет реальным словом намного меньше чем 10-12.

У нас две проблемы:

* Во-первых, у нас нет четкой модели для неизвестных слов. Мы говорим "неизвестное слово", но не различаем более вероятные неизвестные слова и менее вероятные неизвестные слова. Ну, например, вероятнее ли 8-буквенное неизвестное слово чем 20-буквенное неизвестное слово?

* Во-вторых, мы не берем в расчет информацию из частей неизвестных слов. Например, "сверхзвуковой" явно должно быть более вероятным чем "рыбркле".

Для нашего следующего подхода мы используем идеи метода Гуда - Тьюринга. Он оценивает вероятности слов, не встретившихся в нашем Counter'е, на основании вероятностей слов, встретившихся единожды (Можно туда же подключить вероятности для встретившихся 2 раза и т.д.).

Итак, сколько слов встретилось 1 раз в COUNTS? (В COUNTS1 ни одного такого слова нет.) И какие длины у этих слов? Давайте посмотрим:


In [ ]:
singletons = (w for w in COUNTS if COUNTS[w] == 1)
lengths = list(map(len, singletons))
Counter(lengths).most_common()

In [ ]:
matplotlib.pyplot.hist(lengths, bins=len(set(lengths)));

**Анализ:** Длины таких слов распределены похоже на нормальное распределение :)

In [ ]:
def pdist_good_turing_hack(counter, onecounter, base=1/33., prior=1e-8):
    """Вероятность слова при условии данных из счетчика.
    Для неизвестных слов, смотрим на слова, встретившиеся единожды из onecounter, 
    вероятность выбираем, основываясь на длине.
    Воспользуемся идеей метода Гуда-Тьюринга(полностью мы его здесь не реализуем).
    prior -добавочный фактор, который сделает неизвестные слова менее вероятными.
    base -то, насколько мы уменьшаем вероятность за длину слова больше максимального."""
    N = sum(list(counter.values()))
    N2 = sum(list(onecounter.values()))
    lengths = list(map(len, [w for w in onecounter if onecounter[w] == 1]))
    ones = Counter(lengths)
    longest = max(ones)
    return (lambda word: 
            counter[word] / N if (word in counter) 
            else prior * (ones[len(word)] / N2 or 
                          ones[longest] / N2 * base ** (len(word)-longest)))
#Переопределим P1w
P1w = pdist_good_turing_hack(COUNTS1, COUNTS)

In [ ]:
segment.cache.clear()


# THE END

У Норвига в статье код занимает 21 строчку: 


In [ ]:
import re, collections
def words(text):
    return re.findall('[а-ё]+', text.lower())
def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model
    
with codecs.open('words.txt', 'r', encoding = 'windows 1251') as file:
    TEXT = file.read().replace('\n', ' ')

NWORDS = train(words(TEXT))
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
def edits1(word):
    n = len(word)
    return set( [word[0:i]+word[i+1:] for i in range(n)] +                      # deletion
                [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)] +   # transposition
                [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet] +    # alteration
                [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet])    # insertion
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)
def known(words): return set(w for w in words if w in NWORDS)
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=lambda w: NWORDS[w])


Использовать этот код нужно следующим образом –



In [ ]:
to_correct = tokens('Саничка не хочит в шшклу')

for word in to_correct:
    print(correct(word), end = ' ') 

Пусть дано слово, будем пытаться отыскать слово, в котором с наибольшей вероятностью исправлены допущенные ошибки (если ошибок нет, то таким словом будет данное). Разумеется, мы не сможем гарантировать 100% исправления всех ошибок. (Например, если нам дано слово «пак», то правильным будет слово «паз» или «парк» ?), именно поэтому мы используем вероятностный (или другими словами стохастический) подход. 
Будем говорить, что мы пытаемся выбрать такое слово c из всех возможных слов-исправлений, что вероятность появления именно слова c при данном слове w будет максимальна:

`argmaxc P(c|w)`


Согласно теореме Байеса - выражение, записанное выше, эквивалентно следующему выражению:

`argmaxc P(w|c) P(c) / P(w)`

Поскольку P(w) одинакова для всех c мы можем отбросить P(w), что даст нам:

`argmaxc P(w|c) P(c)`

--------------------------------------------------------------------------------------
В этом выражении присутствуют три части.  
Справа налево:

***P(c)*** – вероятность появления слова c (частотность употребления c). Эта вероятность обусловлена самим языком (точнее моделью языка). Иначе говоря, P(c) определяет как часто c встречается в текстах на русском языке. P(«превед») будет достаточно высока, тогда как P(«благоденствовать») будет меньше, а P(«ыгввыцшы») будет около нуля.

***P(w|c)*** – вероятность того, что автор опечатался и написал w, хотя имел в виду c. По сути дела эта вероятность обусловлена частотностью тех или иных ошибок в языке (и называется моделью ошибок языка).

***argmaxc*** – оператор, перебирающий все возможные c в поиске наиболее (вероятнее всего) подходящего из них (т.е. данный оператор ищет такое допустимое c, которе максимизирует условную вероятность появления w при данном c).

Может возникнуть очевидный вопрос – зачем мы преобразовали простое выражение «argmaxc P(c|w)» с помощью какого-то Байеса в более сложное выражение, в котором используются аж две языковые модели, вместо одной? Дело в том, что P(c|w) учитывает в себе сразу обе языковых модели, поэтому очевидно, что проще выделить эти модели и работать с ними по отдельности. Предположим у нас есть слово с опечаткой – «езать», это может быть как «ехать», так и «резать». Для какого из исправлений P(c|w) будет максимально ? Оба исправления имеют примерно одинаковую частотность в русском языке. Хорошо допустим «х» и «з» близко расположены в русской раскладке клавиатуры и это повышает вероятность варианта «ехать», но это не повод, чтоб отбрасывать «резать», ведь «е» и «р» тоже близки. Поэтому лучше не рассматривать P(c|w) как единую величину, ибо нам приходится учитывать и частность исправления c и вероятность исправления c для данной опечатки в w. Удобнее работать с этими двумя вероятностями по отдельности.

----------------------------------------------------------------------------------------------

нужно продолжить описание 